In [10]:
# This strategy assumes predictible returns based on last X number of bars and the relation between O,H,L,C.

In [11]:
import datetime
import pandas as pd
import tstables  
import tables 
import _create_research_data as bt_input
import numpy as np

In [12]:
symbol = 'EUR_USD'
granularity = '1H'

filename = '{}_{}.hdf'.format(symbol, granularity)
df = bt_input.read_hdf_file(filename)
df = df[['bid_o', 'bid_h', 'bid_l', 'bid_c', 'volume']]
df.columns = ['open', 'high', 'low', 'close', 'volume']
#df.head()

In [13]:
# Add same columns as a ratio of various inputs
'''
cols = []

lookback = 12
for i in range(1,lookback+1):
    columnname = 'C_C_' + str(i)    
    df[columnname] =  df['close'] / df['close'].shift(i)
    cols.append(columnname)
    
    columnname = 'C_H_' + str(i)    
    df[columnname] =  df['close'] / df['high'].shift(i)
    cols.append(columnname)
    
    columnname = 'C_L_' + str(i)    
    df[columnname] =  df['close'] / df['low'].shift(i)
    cols.append(columnname)
    
    columnname = 'V_V_' + str(i)    
    df[columnname] =  df['volume'] / df['volume'].shift(i)
    cols.append(columnname)
'''
'''
# Add same columns but convert to binary input
cols = []

lookback = 24
for i in range(1,lookback+1):
    columnname = 'b_C_C_' + str(i)    
    df[columnname] = np.sign( df['close'] / df['close'].shift(i) ) 
    cols.append(columnname)
    
    columnname = 'b_C_H_' + str(i)    
    df[columnname] = np.sign( df['close'] / df['high'].shift(i) )
    cols.append(columnname)
    
    columnname = 'b_C_L_' + str(i)    
    df[columnname] = np.sign( df['close'] / df['low'].shift(i) )
    cols.append(columnname)
    
    columnname = 'b_V_V_' + str(i)    
    df[columnname] = np.sign( df['volume'] / df['volume'].shift(i) )
    cols.append(columnname)
'''

# Add same columns but convert to binary input
cols = []

lookback = 24
for i in range(1,lookback+1):
    columnname = 'log_C_C_' + str(i)    
    df[columnname] = np.log( df['close'] / df['close'].shift(i) ) 
    cols.append(columnname)
    
    columnname = 'log_C_H_' + str(i)    
    df[columnname] = np.log( df['close'] / df['high'].shift(i) )
    cols.append(columnname)
    
    columnname = 'log_C_L_' + str(i)    
    df[columnname] = np.log( df['close'] / df['low'].shift(i) )
    cols.append(columnname)
    
    columnname = 'log_V_V_' + str(i)    
    df[columnname] = np.log( df['volume'] / df['volume'].shift(i) )
    cols.append(columnname)

'''
# Add same columns but convert to binary input
cols = []

lookback = 24
for i in range(1,lookback+1):
    columnname = 'b_log_C_C_' + str(i)    
    df[columnname] = n.sign( np.log( df['close'] / df['close'].shift(i) ) )
    cols.append(columnname)
    
    columnname = 'b_log_C_H_' + str(i)    
    df[columnname] = n.sign( np.log( df['close'] / df['high'].shift(i) ) )
    cols.append(columnname)
    
    columnname = 'b_log_C_L_' + str(i)    
    df[columnname] = n.sign( np.log( df['close'] / df['low'].shift(i) ) )
    cols.append(columnname)
    
    columnname = 'b_log_V_V_' + str(i)    
    df[columnname] = n.sign( np.log( df['volume'] / df['volume'].shift(i) ) )
    cols.append(columnname)
'''

# Add response variable

df['return'] = df['close'] / df['close'].shift(1) - 1
df['b_return'] = np.sign( df['return'] )
df['log_return'] = np.log( df['close'] / df['close'].shift(1) )
df['b_log_return'] = np.sign( df['log_return'] )

df['Y'] =  df['log_return'].shift(-1)
df['b_Y'] =  df['b_log_return'].shift(-1)

df = df.dropna()

#df.to_excel('df.xlsx')

In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix


In [15]:
X = df[cols].values
X_scaled = preprocessing.scale(X)
y = df['b_Y'].values

In [17]:
lr = LogisticRegression(solver='newton-cg')
lr.fit(X_scaled, y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='newton-cg', tol=0.0001, verbose=0,
                   warm_start=False)

In [18]:
#Accuracy is defined as:
#(fraction of correct predictions): correct predictions / total number of data points

# Use score method to get accuracy of model
df['prediction'] = lr.predict(X_scaled)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(lr.score(X_scaled, y)))


Accuracy of logistic regression classifier on test set: 0.53


In [19]:
confusion_matrix = confusion_matrix(df['b_log_return'], df['prediction'])
print(confusion_matrix)

[[11113     0 20061]
 [  135     0   211]
 [19312     0 11933]]


In [20]:
the training accuracy
accuracy_score(df['prediction'], df['b_log_return'])



0.3671791603600733

In [21]:
df['b_log_return'].unique()

array([-1.,  1.,  0.])

In [ ]:
hits = np.sign(data['returns'].iloc[lags:] *
               data['prediction'].iloc[lags:]
              ).value_counts()


In [ ]:
df['strategy'] = df['prediction'] * df['return']
df[['return', 'strategy']].cumsum().apply(np.exp)